#### Notebook to map ridership % change to Bay Area Zipcodes 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import requests
import folium  #needed for interactive map
from tqdm import tqdm 
import json

#### Load data 

In [7]:
# Bart Raw Data
t2021 = pd.read_csv('BART_DATASET/date-hour-soo-dest-2021.csv', names=['Day', 'Hour', 'Origin', 'Destination', 'Trip Count'])
t2020 = pd.read_csv('BART_DATASET/date-hour-soo-dest-2020.csv', names=['Day', 'Hour', 'Origin', 'Destination', 'Trip Count'])
t2019 = pd.read_csv('BART_DATASET/date-hour-soo-dest-2019.csv', names=['Day', 'Hour', 'Origin', 'Destination', 'Trip Count'])
# t2018 = pd.read_csv('BART_DATASET/date-hour-soo-dest-2018.csv', names=['Day', 'Hour', 'Origin', 'Destination', 'Trip Count'])
# t2017 = pd.read_csv('BART_DATASET/date-hour-soo-dest-2017.csv', names=['Day', 'Hour', 'Origin', 'Destination', 'Trip Count'])
# t2016 = pd.read_csv('BART_DATASET/date-hour-soo-dest-2016.csv', names=['Day', 'Hour', 'Origin', 'Destination', 'Trip Count'])
# t2015 = pd.read_csv('BART_DATASET/date-hour-soo-dest-2015.csv', names=['Day', 'Hour', 'Origin', 'Destination', 'Trip Count'])
# t2014 = pd.read_csv('BART_DATASET/date-hour-soo-dest-2014.csv', names=['Day', 'Hour', 'Origin', 'Destination', 'Trip Count'])

# Cleaned data; additional information necessary 
stat_names = pd.read_csv('BART_STATIONS_ABBREVIATIONS_ZIPCODES.txt', index_col='Abbr')
zip_dict = {i:stat_names['Zipcode'][i] for i in stat_names.index}
# full_df = pd.read_csv('BART_cleaned_updated_dataset.csv', index_col='Day')

In [8]:
realtor = pd.read_csv('realtor_perc_change_df.csv')

#### Clean SF Geo-JSon File

In [10]:
#clean original sf data 
with open('sfbayarea-geojson.json') as jsonFile: 
    data = json.load(jsonFile)
tmp = data

# only include zipcodes that are in the station zipcode list 
# this would probably have to be changed for the realtor zipcodes as there are probably some zipcodes that do not 
# overlap
geozips = []
for i in range(len(tmp['features'])):
    if int(tmp['features'][i]['properties']['ZIP']) in list(realtor['postal_code']):
        geozips.append(tmp['features'][i])
        
new_json = dict.fromkeys(['type', 'features'])
new_json['type'] = 'FeatureCollection'
new_json['features'] = geozips

open('sfbayarea-updated-realtor-geojson.json', 'w').write(
    json.dumps(new_json, sort_keys=True, indent=4, separators=(',', ': ' )))

320832

#### Functions for analysis

In [33]:
def add_year(df):
    if 'Year' not in df.columns:
        df['Day'] = pd.to_datetime(df['Day'])
        df.set_index('Day', inplace=True)
        year = (df.index).year
        df['Year'] = year
    

def add_zip(df):
    global zip_dict
    if 'Origin_Zipcode' not in df.columns: 
        og_zip = []
        for station in tqdm(df.Origin): 
            og_zip.append(zip_dict[station])
        df['Origin_Zipcode'] = og_zip

        dest_zip = []
        for station in tqdm(df.Destination):
            dest_zip.append(zip_dict[station])
        df['Destination_Zipcode'] = dest_zip
    
def perc_change(df1, df2, groupby:bool, origin:bool=True):
    """
    df1: dataframe of base year (older)
    df2: dataframe of new year
    groupby: grouping by stations (True) or zipcodes (False)
    origin: True/False, true: looking at origin stations, false: looking at destination stations
    """
    global zip_dict
    add_year(df1)
    add_year(df2)
    add_zip(df1)
    add_zip(df2)
    if groupby & origin: #groupby origin stations
        col = 'Origin'
    elif groupby and not origin: #groupby destination stations
        col = 'Destination'
    elif not groupby and origin: #groupby origin zipcodes
        col = 'Origin_Zipcode'
    elif not groupby and not origin: #groupby destination zipcodes
        col = 'Destination_Zipcode'
    
    old_sums = df1.groupby(col).sum().sort_values(by='Trip Count', ascending=False)['Trip Count']
    new_sums = df2.groupby(col).sum().sort_values(by = 'Trip Count', ascending = False)['Trip Count']
#     try: 
#         if df1 is t2014 and groupby: 
#             del old_sums['OAKL']
#             del old_sums[zip_dict['OAKL']]
#     except NameError:
#         pass 
    
#     try: 
    if df2 is t2020 or df2 is t2021 and not groupby: 
        del new_sums[zip_dict['BERY']]
        del new_sums[zip_dict['MLPT']]
    elif df2 in [t2020, t2021] and groupby: 
        del new_sums['BERY']
        del new_sums['MLPT']
    elif df1 is t2020 and groupby: 
        del old_sums['BERY']
        del old_sums['MLPT']
    elif df1 is t2021 and not groupby: 
        del old_sums[zip_dict['BERY']]
        del old_sums[zip_dict['MLPT']]
#     except NameError:
#         pass 
    
    new_sums = new_sums.reindex(old_sums.index)
    
    diff_df = pd.DataFrame([])
    diff_df[col] = old_sums.index
    name = '%change ' + str(df1['Year'][0]) + 'to ' + str(df2['Year'][0])
    diff_df[name] = ((new_sums - old_sums)/old_sums).values * 100
    diff_df = diff_df.sort_values(by=name, ascending=False)
    return diff_df, old_sums, new_sums

def create_map(df1, df2, groupby=False, origin=True, add_text = ''):
    #san francisco coordinates 
    """
    df1, df2: defined same as previously. dataframes of the years we want to compare. df1 older. 
    Table: dataframe 
    add_text: title
    """
    add_zip(df1)
    add_zip(df2)
    table = perc_change(df1, df2, groupby, origin)[0]
    zips = table.columns[0]
    mapped_feature = table.columns[1]
    table[zips] = table[zips].astype(str)
    sf_coords = [37.7792808, -122.4192363]
    with open('sfbayarea-updated-geojson.json') as jsonFile: 
        data = json.load(jsonFile)
    sf_geo = data
    m = folium.Map(location=sf_coords, zoom_start=13, tiles="OpenStreetMap")
    
    
    folium.Choropleth(geo_json_data,
               data = table,
            key_on = 'feature.properties.ZIP',
            columns = [zips, mapped_feature],
            fill_color = 'RdYlGn'
          ).add_to(m)
    m.save(outfile = mapped_feature + '_map.html')
    return m

#### add zipcodes

In [34]:
add_zip(t2019)
add_zip(t2020)
add_zip(t2021)

In [36]:
## % change in ridership 2019 to 2021
create_map(t2019, t2021)